In [1]:
import json
import os
import pandas as pd
import SimpleITK as sitk
import torch
import numpy as np
import scipy.ndimage
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import skimage.transform
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score,recall_score
from plt_metrics import plot_roc_curve,plot_confusion_matrix,plot_f1_curve
from plt_metrics import save_accuracy_auc_plot,save_train_val_accuracy_plot
from plt_metrics import plot_loss_curve

In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import SimpleITK as sitk
import numpy as np


class NiiDataset(Dataset):
    def __init__(self, normal_dir, tumor_dir, target_size=(10, 224, 224),transform=None):
        self.normal_dir = normal_dir
        self.tumor_dir = tumor_dir
        self.target_size = target_size
        self.transform = transform

        # Gather file paths
        self.normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith(".nii.gz")]
        self.tumor_files = [os.path.join(tumor_dir, f) for f in os.listdir(tumor_dir) if f.endswith(".nii.gz")]

        # Combine and assign labels (0 for normal, 1 for tumor)
        self.data = [(file, 0) for file in self.normal_files] + [(file, 1) for file in self.tumor_files]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path, label = self.data[idx]

        image = sitk.ReadImage(file_path)
        image_array = sitk.GetArrayFromImage(image) 
        reshaped_array = np.transpose(image_array, (2, 1, 0))

        img_array = self.resize_image(reshaped_array)
        img_array = img_array.astype(np.float32)

             # Normalize and add channel dimension
        img_array = torch.tensor(img_array, dtype=torch.float32)
        img_array = img_array.unsqueeze(0)  # Add channel dimension

        label = torch.tensor(label, dtype=torch.long)
        
        return img_array, label
    
    def resize_image(self, img_array):
        depth,height, width = img_array.shape
        target_depth, target_height, target_width = self.target_size
            
            # Calculate scale factors for each dimension
        depth_scale = target_depth / depth
        height_scale = target_height / height
        width_scale = target_width / width
            
            # Use scipy.ndimage.zoom to resize the 3D image array

        resized_array = skimage.transform.resize(img_array,
                                         output_shape=(target_depth, target_height, target_width),
                                         mode='reflect',  # 边界处理模式
                                         anti_aliasing=True)  # 使用抗锯齿
        
        return resized_array

    
train_normal_dir = "F:/Dataset/Train_normal_more_cases/manifest-1617905855234/Breast-Cancer-Screening-DBT/3d_train"
train_tumor_dir = "F:/Dataset/Train_tumor/manifest-1617905855234/Breast-Cancer-Screening-DBT/3d_slices_nii_3D_10_5P"         # Replace with your tumor folder path

train_dataset = NiiDataset(train_normal_dir, train_tumor_dir, target_size=(10, 224, 224))
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

val_normal_dir = "F:/Dataset/Train_normal_more_cases/manifest-1617905855234/Breast-Cancer-Screening-DBT/3d_val" 
val_tumor_dir = "F:/Dataset/Val/manifest-1617905855234/Breast-Cancer-Screening-DBT/3d_slices_nii_3D_10_5P"         # Replace with your tumor folder path

val_dataset = NiiDataset(train_normal_dir, train_tumor_dir, target_size=(10, 224, 224))
val_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)


In [ ]:
img_array, label = next(iter(train_dataloader))
print(img_array.shape, label)

In [4]:
from run_DenseNet import train,validate,main_training_loop,save_loss_to_file

In [ ]:
from model.resnet import ResNet3D
from model.resnet import BasicBlock

model = ResNet3D(BasicBlock, [2, 2, 2, 2], num_classes=2)

metric_folder = '3D_resnet_normal_tumor'

trained_model = main_training_loop(model, train_dataloader, val_dataloader, num_epochs=50, lr=0.0001, device='cuda',metric_folder = metric_folder)